In [3]:
using Pkg; Pkg.activate("C:\\Users\\KarlH\\Desktop\\ComputationalPhysicsMaster\\Code\\OrbitalNeuralMethods")
using Revise
using OrbitalNeuralMethods

using BenchmarkTools: @btime
import OrbitalNeuralMethods as onm

  Activating project at `C:\Users\KarlH\Desktop\ComputationalPhysicsMaster\Code\OrbitalNeuralMethods`
┌ Info: Precompiling OrbitalNeuralMethods [6de8678a-dd1c-461e-a987-24da1d9562b4]
└ @ Base loading.jl:1662


In [10]:
n = 2

ω = 0.25
ham = HOCoulomb(ω, shielding = 0.25)
#ham = HarmonicOscillator(ω)

l = 10
basis = SpinBasis(HOBasis(l, ω));

In [13]:
grid = [x for x in range(-10, stop = 10, length = 2001)]
system = System(n, basis, grid, ham);

mixer = Alpha(0.5)
rhf = setup_RHF(system, mixer);
rhf = compute_ground_state!(rhf, max_iter = 1000, tol = 1e-10, verbose=0);

In [14]:
wf = Slater(rhf)

metro = Importance(equils = 1e5, samples = 1048576, step = 0.01);
#metro = Metropolis(equils = 1e4, samples = 1e5, step = 0.01)

In [16]:
@time block_result = blocking(wf, ham, metro, nthreads = 1)
block_result.E

LoadError: TaskFailedException

[91m    nested task error: [39mMethodError: no method matching OrbitalNeuralMethods.Metro_Muts(::Slater, ::Importance)
    [0mClosest candidates are:
    [0m  OrbitalNeuralMethods.Metro_Muts(::Any) at C:\Users\KarlH\Desktop\ComputationalPhysicsMaster\Code\OrbitalNeuralMethods\src\VMC\Walkers\muts.jl:25
    Stacktrace:
     [1] [0m[1mEquilWalker[22m[0m[1m([22m[90mwf[39m::[0mSlater, [90mmetro[39m::[0mImportance[0m[1m)[22m
    [90m   @ [39m[35mOrbitalNeuralMethods[39m [90mC:\Users\KarlH\Desktop\ComputationalPhysicsMaster\Code\OrbitalNeuralMethods\src\VMC\Walkers\[39m[90m[4mwalker.jl:17[24m[39m
     [2] [0m[1mmacro expansion[22m
    [90m   @ [39m[90mC:\Users\KarlH\Desktop\ComputationalPhysicsMaster\Code\OrbitalNeuralMethods\src\VMC\[39m[90m[4msteps.jl:10[24m[39m[90m [inlined][39m
     [3] [0m[1m#201#threadsfor_fun#45[22m
    [90m   @ [39m[90m.\[39m[90m[4mthreadingconstructs.jl:84[24m[39m[90m [inlined][39